In [38]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast


In [39]:
from skforecast.drift_detection import RangeDriftDetector
from pprint import pprint
from copy import deepcopy

In [40]:
y = pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D'), name="y")
series_df = pd.DataFrame(
    {
        'series_1': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
        'series_2': pd.Series(np.random.rand(5000), index=pd.date_range(start='2000-01-01', periods=5000, freq='D')),
    }
)
series_df_long = series_df.stack().to_frame().rename(columns={0: 'y'}).swaplevel(0,1).sort_index()
exog = pd.Series(np.random.rand(5000), index=y.index, name="temp")
exog_df = pd.DataFrame(index=y.index)
for i in range(5):
    exog_df[f"feature_{i}"] = np.random.rand(5000)
for i in range(5):
    exog_df[f"cat_feature_{i}"] = np.random.choice(["A", "B", "C"], 5000)


series_dict = series_df.to_dict(orient='series')
exog_dict = {'series_1': exog_df, 'series_2': exog_df}
exog_df_long = [
    exog_df.copy().assign(series_id='series_1'),
    exog_df.copy().assign(series_id='series_2')
]
exog_df_long = pd.concat(exog_df_long).set_index('series_id', append=True).reorder_levels(['series_id', None]).sort_index()

# FIT

In [41]:
range_detector = RangeDriftDetector()
range_detector.fit(y)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'y': (np.float64(0.0004567519813596377), np.float64(0.9992564615163493))}
['y']


In [42]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.00010544681752122642),
              np.float64(0.9999093350287258)),
 'series_2': (np.float64(5.664282451411928e-05),
              np.float64(0.9999445380653009))}
['series_1', 'series_2']


In [43]:
range_detector

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.00010544681752122642), np.float64(0.9999093350287258)), 'series_2': (np.float64(5.664282451411928e-05), np.float64(0.9999445380653009))} 
Exogenous value ranges: None 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: None 

In [44]:
range_detector = RangeDriftDetector()
range_detector.fit(series_dict)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.00010544681752122642),
              np.float64(0.9999093350287258)),
 'series_2': (np.float64(5.664282451411928e-05),
              np.float64(0.9999445380653009))}
['series_1', 'series_2']


In [45]:
range_detector = RangeDriftDetector()
range_detector.fit(series_df_long)
pprint(range_detector.series_values_range_)
print(range_detector.series_names_in_)

{'series_1': (np.float64(0.00010544681752122642),
              np.float64(0.9999093350287258)),
 'series_2': (np.float64(5.664282451411928e-05),
              np.float64(0.9999445380653009))}
['series_1', 'series_2']


In [46]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'temp': (np.float64(0.00010237385364775253), np.float64(0.9997389320314732))}
['temp']


In [47]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'cat_feature_0': {'C', 'B', 'A'},
 'cat_feature_1': {'B', 'A', 'C'},
 'cat_feature_2': {'B', 'A', 'C'},
 'cat_feature_3': {'C', 'B', 'A'},
 'cat_feature_4': {'C', 'B', 'A'},
 'feature_0': (np.float64(0.00021278137512736883),
               np.float64(0.9999720550274622)),
 'feature_1': (np.float64(0.0003363650385935779),
               np.float64(0.9999300771485099)),
 'feature_2': (np.float64(0.0003969699474314359),
               np.float64(0.9993336791016134)),
 'feature_3': (np.float64(0.00025741409341029087),
               np.float64(0.9997747312007411)),
 'feature_4': (np.float64(0.00011645607588495199),
               np.float64(0.9998120592344611))}
['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'cat_feature_0', 'cat_feature_1', 'cat_feature_2', 'cat_feature_3', 'cat_feature_4']


In [48]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_dict)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'B', 'A', 'C'},
              'cat_feature_3': {'C', 'B', 'A'},
              'cat_feature_4': {'C', 'B', 'A'},
              'feature_0': (np.float64(0.00021278137512736883),
                            np.float64(0.9999720550274622)),
              'feature_1': (np.float64(0.0003363650385935779),
                            np.float64(0.9999300771485099)),
              'feature_2': (np.float64(0.0003969699474314359),
                            np.float64(0.9993336791016134)),
              'feature_3': (np.float64(0.00025741409341029087),
                            np.float64(0.9997747312007411)),
              'feature_4': (np.float64(0.00011645607588495199),
                            np.float64(0.9998120592344611))},
 'series_2': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'B', 'A'

In [49]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog_df_long)
pprint(range_detector.exog_values_range_)
print(range_detector.exog_names_in_)

{'series_1': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'B', 'A', 'C'},
              'cat_feature_3': {'C', 'B', 'A'},
              'cat_feature_4': {'C', 'B', 'A'},
              'feature_0': (np.float64(0.00021278137512736883),
                            np.float64(0.9999720550274622)),
              'feature_1': (np.float64(0.0003363650385935779),
                            np.float64(0.9999300771485099)),
              'feature_2': (np.float64(0.0003969699474314359),
                            np.float64(0.9993336791016134)),
              'feature_3': (np.float64(0.00025741409341029087),
                            np.float64(0.9997747312007411)),
              'feature_4': (np.float64(0.00011645607588495199),
                            np.float64(0.9998120592344611))},
 'series_2': {'cat_feature_0': {'C', 'B', 'A'},
              'cat_feature_1': {'B', 'A', 'C'},
              'cat_feature_2': {'B', 'A'

# Predict

In [50]:
range_detector = RangeDriftDetector()
range_detector.fit(y, exog=exog)
pprint(range_detector.series_values_range_)
pprint(range_detector.exog_values_range_)


{'y': (np.float64(0.0004567519813596377), np.float64(0.9992564615163493))}
{'temp': (np.float64(0.00010237385364775253), np.float64(0.9997389320314732))}


In [51]:
last_window_valid = y.tail(100).copy()
last_window_invalid = y.tail(100).copy()
last_window_invalid.iloc[5] = 10_000    
range_detector.predict(last_window=last_window_valid)
range_detector.predict(last_window=last_window_invalid)
range_detector


All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'y' has one or more values outside the range seen during training [0.00046,          │
│ 0.99926]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'y' has one or more values outside the range seen during training [0.00046, 0.99926]. 
'y' has one or more values outside the range seen during training [0.00046, 0.99926]. 



RangeDriftDetector 
Series value ranges: {'y': (np.float64(0.0004567519813596377), np.float64(0.9992564615163493))} 
Exogenous value ranges: {'temp': (np.float64(0.00010237385364775253), np.float64(0.9997389320314732))} 
Fitted series: ['y'] 
Fitted exogenous: ['temp'] 

In [52]:
range_detector.fit(series_df)
last_window_valid_df = series_df.tail(100).copy()
last_window_invalid_df = series_df.tail(100).copy()
last_window_invalid_df.iloc[1, [0, 1]] = 10_000
range_detector.predict(last_window=last_window_valid_df)
range_detector.predict(last_window=last_window_invalid_df)
range_detector

All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1' has one or more values outside the range seen during training [0.00011,   │
│ 0.99991]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2' has one or more values outside the range seen during training [0.00006,   │
│ 0.99994]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'series_2' has one or more values outside the range seen during training [0.00006, 0.99994]. 
'series_2' has one or more values outside the range seen during training [0.00006, 0.99994]. 



RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.00010544681752122642), np.float64(0.9999093350287258)), 'series_2': (np.float64(5.664282451411928e-05), np.float64(0.9999445380653009))} 
Exogenous value ranges: None 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: None 

In [53]:
range_detector.fit(series_dict)
last_window_valid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
last_window_invalid_dict = {key: val.tail(100).copy() for key, val in series_dict.items()}
for key in last_window_invalid_dict.keys():
    last_window_invalid_dict[key].iloc[1] = 10_000
range_detector.predict(last_window=last_window_valid_dict)
range_detector.predict(last_window=last_window_invalid_dict)
range_detector

All series and exogenous variables are within the training range.
'series_2' has one or more values outside the range seen during training [0.00006, 0.99994]. 
'series_2' has one or more values outside the range seen during training [0.00006, 0.99994]. 



RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.00010544681752122642), np.float64(0.9999093350287258)), 'series_2': (np.float64(5.664282451411928e-05), np.float64(0.9999445380653009))} 
Exogenous value ranges: None 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: None 

In [54]:
range_detector.fit(series_df_long)
last_window_valid_df_long= series_df_long.groupby(level=0).tail(100).copy()
last_window_invalid_df_long = series_df_long.groupby(level=0).tail(100).copy()
for key in last_window_invalid_df_long.index.get_level_values(0).unique():
    last_window_invalid_df_long.loc[key, pd.Timestamp("2013-06-02")] = 10_000
range_detector.predict(last_window=last_window_valid_df_long)
range_detector.predict(last_window=last_window_invalid_df_long)

All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1' has one or more values outside the range seen during training [0.00011,   │
│ 0.99991]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2' has one or more values outside the range seen during training [0.00006,   │
│ 0.99994]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'series_2' has one or more values outside the range seen during training [0.00006, 0.99994]. 
'series_2' has one or more values outside the range seen during training [0.00006, 0.99994]. 



(True, ['series_1', 'series_2'], [])

In [55]:
range_detector.fit(series_df, exog=exog)
print(range_detector)
exog_valid = exog.tail(100).copy()
exog_invalid = exog.tail(100).copy()
exog_invalid.iloc[5] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.00010544681752122642), np.float64(0.9999093350287258)), 'series_2': (np.float64(5.664282451411928e-05), np.float64(0.9999445380653009))} 
Exogenous value ranges: {'temp': (np.float64(0.00010237385364775253), np.float64(0.9997389320314732))} 
Fitted series: ['series_1', 'series_2'] 
Fitted exogenous: ['temp'] 

All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'temp' has one or more values outside the range seen during training [0.00010,       │
│ 0.99974]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'temp': 'temp' has one or more values outside the range seen during training [0.00010, 0.99974]. 
'temp': 'temp' has one or more values outside the range seen during training [0.00010, 0.99974]. 



(True, [], ['temp'])

In [56]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0' has one or more values outside the range seen during training [0.00021,  │
│ 0.99997]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1' has one or more values outside the range seen during training [0.00034,  │
│ 0.99993]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'feature_1': 'feature_1' has one or more values outside the range seen during training [0.00034, 0.99993]. 
'feature_1': 'feature_1' has one or more values outside the range seen during training [0.00034, 0.99993]. 



(True, [], ['feature_0', 'feature_1'])

In [57]:
range_detector = RangeDriftDetector()
range_detector.fit(series=y, exog=exog_df)
range_detector
exog_valid = exog_df.tail(100).copy()
exog_invalid = exog_df.tail(100).copy()
exog_invalid.iloc[5, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)

All series and exogenous variables are within the training range.


╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_0' has one or more values outside the range seen during training [0.00021,  │
│ 0.99997]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'feature_1' has one or more values outside the range seen during training [0.00034,  │
│ 0.99993]. This may affect the accuracy of the predictions.                           │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'feature_1': 'feature_1' has one or more values outside the range seen during training [0.00034, 0.99993]. 
'feature_1': 'feature_1' has one or more values outside the range seen during training [0.00034, 0.99993]. 



(True, [], ['feature_0', 'feature_1'])

In [58]:
range_detector = RangeDriftDetector()

range_detector.fit(series=series_dict, exog=exog_dict)
print(range_detector)
exog_valid = deepcopy(exog_dict)
exog_invalid = deepcopy(exog_dict)
for key in exog_invalid.keys():
    
    exog_invalid[key].iloc[1, [0,1]] = 10_000
range_detector.predict(exog=exog_valid)
range_detector.predict(exog=exog_invalid)
range_detector

RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.00010544681752122642), np.float64(0.9999093350287258)), 'series_2': (np.float64(5.664282451411928e-05), np.float64(0.9999445380653009))} 
Exogenous value ranges: {'series_1': {'feature_0': (np.float64(0.00021278137512736883), np.float64(0.9999720550274622)), 'feature_1': (np.float64(0.0003363650385935779), np.float64(0.9999300771485099)), 'feature_2': (np.float64(0.0003969699474314359), np.float64(0.9993336791016134)), 'feature_3': (np.float64(0.00025741409341029087), np.float64(0.9997747312007411)), 'feature_4': (np.float64(0.00011645607588495199), np.float64(0.9998120592344611)), 'cat_feature_0': {'C', 'B', 'A'}, 'cat_feature_1': {'B', 'A', 'C'}, 'cat_feature_2': {'B', 'A', 'C'}, 'cat_feature_3': {'C', 'B', 'A'}, 'cat_feature_4': {'C', 'B', 'A'}}, 'series_2': {'feature_0': (np.float64(0.00021278137512736883), np.float64(0.9999720550274622)), 'feature_1': (np.float64(0.0003363650385935779), np.float64(0.99993007714850

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_0' has one or more values outside the range seen during         │
│ training [0.00021, 0.99997]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_1': 'feature_1' has one or more values outside the range seen during         │
│ training [0.00034, 0.99993]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_0' has one or more values outside the range seen during         │
│ training [0.00021, 0.99997]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── FeatureOutOfRangeWarning ──────────────────────────────╮
│ 'series_2': 'feature_1' has one or more values outside the range seen during         │
│ training [0.00034, 0.99993]. This may affect the accuracy of the predictions.        │
│                                                                                      │
│ Category : skforecast.exceptions.FeatureOutOfRangeWarning                            │
│ Location :                                                                           │
│ /home/joaquin/Documents/GitHub/skforecast/skforecast/drift_detection/_range_drift.py │
│ :185                                                                                 │
│ Suppress : warnings.simplefilter('ignore', category=FeatureOutOfRangeWarning)        │
╰──────────────────────────────────────────────────────────────────────────────────────╯

'series_2': 'feature_1' has one or more values outside the range seen during training [0.00034, 0.99993]. 
'series_2': 'feature_1' has one or more values outside the range seen during training [0.00034, 0.99993]. 



RangeDriftDetector 
Series value ranges: {'series_1': (np.float64(0.00010544681752122642), np.float64(0.9999093350287258)), 'series_2': (np.float64(5.664282451411928e-05), np.float64(0.9999445380653009))} 
Exogenous value ranges: {'series_1': {'feature_0': (np.float64(0.00021278137512736883), np.float64(0.9999720550274622)), 'feature_1': (np.float64(0.0003363650385935779), np.float64(0.9999300771485099)), 'feature_2': (np.float64(0.0003969699474314359), np.float64(0.9993336791016134)), 'feature_3': (np.float64(0.00025741409341029087), np.float64(0.9997747312007411)), 'feature_4': (np.float64(0.00011645607588495199), np.float64(0.9998120592344611)), 'cat_feature_0': {'C', 'B', 'A'}, 'cat_feature_1': {'B', 'A', 'C'}, 'cat_feature_2': {'B', 'A', 'C'}, 'cat_feature_3': {'C', 'B', 'A'}, 'cat_feature_4': {'C', 'B', 'A'}}, 'series_2': {'feature_0': (np.float64(0.00021278137512736883), np.float64(0.9999720550274622)), 'feature_1': (np.float64(0.0003363650385935779), np.float64(0.99993007714850

In [59]:
exog_valid['series_1']['feature_1000'] = 10000